In [1]:
import numpy as np
import pandas as pd

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.base import backtest_df_only, order_records_bt
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb
from quantfreedom.strategies.strategy_example import StrategyExample
import requests


mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2


In [2]:
candles = mufex.get_candles_df(
    symbol=symbol, timeframe='5m', candles_to_dl=300)

It took 00 mins and 01 seconds to download 300 candles


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    long_or_short=np.array([LongOrShortType.Long]),
    increase_position_type=np.array(
        [IncreasePositionType.RiskPctAccountEntrySize]),
    risk_account_pct_size=np.array([1]),
    max_equity_risk_pct=np.array([3]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]),
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]),
    trail_sl_by_pct=np.array([1.0]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    risk_reward=np.array([2]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    static_leverage=np.array([0.0]),
    num_candles=np.array([200]),
    entry_size_asset=np.array([0.0]),
    max_trades=np.array([3]),

)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [4]:
strategy = StrategyExample(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    rsi_length=[14],
    rsi_is_below=[50],
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
    candles=candles,
)


Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 1
Total combinations of settings to test: 1

Total candles: 300
Total candles to test: 300


In [5]:
candles_reg.sort_values(['ind_set_idx', 'or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,0,0,24.0,-20.34,16.67,-0.8754,-203.4155,796.5845


In [6]:
order_records = order_records_bt(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
    candles=candles,
    backtest_results=candles_reg,
    backtest_index=0,
)
order_records.timestamp = pd.to_datetime(order_records.timestamp, unit="ms")

In [7]:
for i in range(len(OrderStatus._fields)):
    order_records.replace(
        {"order_status": {i: OrderStatus._fields[i]}}, inplace=True
    )

In [8]:
order_records.order_status.unique()

array(['EntryFilled', 'StopLossFilled', 'TakeProfitFilled'], dtype=object)

In [9]:
entry_filled_df = order_records[order_records["order_status"] == "EntryFilled"][
    ["bar_idx", "entry_price", "average_entry", "liq_price", "sl_price", "tp_price"]
]

entry_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()
avg_entry_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()
liq_price_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()
sl_price_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()
tp_price_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()

for _, value in entry_filled_df.T.items():
    bar_index = int(value["bar_idx"])
    entry_list[bar_index].append(value["entry_price"])
    avg_entry_list[bar_index].append(value["average_entry"])
    liq_price_list[bar_index].append(value["liq_price"])
    sl_price_list[bar_index].append(value["sl_price"])
    tp_price_list[bar_index].append(value["tp_price"])

In [10]:
exit_filled_df = order_records[order_records["order_status"] != "EntryFilled"][
    ["bar_idx", 'order_status', "exit_price"]
]

sl_filled_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()
tp_filled_list = np.reshape(candles["timestamp"], (-1, 1)).tolist()

for _, value in exit_filled_df.T.items():
    bar_index = int(value["bar_idx"])
    if value["order_status"] == "StopLossFilled":
        sl_filled_list[bar_index].append(value["exit_price"])
    else:
        tp_filled_list[bar_index].append(value["exit_price"])

In [17]:
exit_filled_df

,bar_idx,order_status,exit_price
3,52,StopLossFilled,28378.1
6,54,StopLossFilled,28302.3
8,55,StopLossFilled,28246.2
12,94,StopLossFilled,28419.2
15,99,StopLossFilled,28294.5
17,100,StopLossFilled,28262.8
21,130,StopLossFilled,28274.2
23,131,StopLossFilled,28262.0
26,133,StopLossFilled,28252.3
29,135,StopLossFilled,28247.1


In [11]:
rsi = np.reshape(candles['timestamp'], (-1, 1)).tolist()
for index,value in enumerate(rsi):
    if not np.isnan(strategy.rsi[index]):
        value.append(strategy.rsi[index])

In [12]:
entries_info = {
    "entries": entry_list,
    "avg_entries": avg_entry_list,
    "liq_prices": liq_price_list,
    "sl_prices": sl_price_list,
    "tp_prices": tp_price_list,
    "sl_filled": sl_filled_list,
    "tp_filled": tp_filled_list,
    "rsi": rsi,
    "candles": pd.DataFrame(candles).values.tolist(),
}

In [13]:
strategy.rsi[:20]

array([  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
         nan,   nan,   nan,   nan,   nan, 57.99, 55.48, 62.31, 66.74,
       68.97, 74.26])

In [14]:
rsi[:30]

[[1697474100000],
 [1697474400000],
 [1697474700000],
 [1697475000000],
 [1697475300000],
 [1697475600000],
 [1697475900000],
 [1697476200000],
 [1697476500000],
 [1697476800000],
 [1697477100000],
 [1697477400000],
 [1697477700000],
 [1697478000000],
 [1697478300000, 57.99],
 [1697478600000, 55.48],
 [1697478900000, 62.31],
 [1697479200000, 66.74],
 [1697479500000, 68.97],
 [1697479800000, 74.26],
 [1697480100000, 76.22],
 [1697480400000, 64.72],
 [1697480700000, 66.91],
 [1697481000000, 61.74],
 [1697481300000, 65.45],
 [1697481600000, 64.14],
 [1697481900000, 68.78],
 [1697482200000, 74.54],
 [1697482500000, 75.56],
 [1697482800000, 80.3]]

In [16]:
# The POST request to our NightVision server
res = requests.post("http://127.0.0.1:7779/plot", json=entries_info)

# Convert response data to json
returned_data = res.json()

print(returned_data)


{'ok': True}
